# Mask R-CNN - Human pose estimation

Inspect and visualize data loading and pre-processing code.

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import utils
import visualize
from visualize import display_images
import model as modellib
from model import log

%matplotlib inline 

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "mylogs")
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco_humanpose.h5")



## Configurations

Run one of the code blocks below to import and load the configurations to use.

In [2]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = "D:/Eigene Dateien/Dokumente/coco"  # TODO: enter your own path here

## Dataset

In [3]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = "D:/Eigene Dateien/Dokumente/coco"  # TODO: enter value here
# Load dataset
assert config.NAME == "coco"
# Training dataset
# load person keypoints dataset
train_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
train_dataset_keypoints.load_coco(COCO_DIR, "train")
train_dataset_keypoints.prepare()

#Validation dataset
val_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
val_dataset_keypoints.load_coco(COCO_DIR, "val")
val_dataset_keypoints.prepare()

print("Train Keypoints Image Count: {}".format(len(train_dataset_keypoints.image_ids)))
print("Train Keypoints Class Count: {}".format(train_dataset_keypoints.num_classes))
for i, info in enumerate(train_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

print("Val Keypoints Image Count: {}".format(len(val_dataset_keypoints.image_ids)))
print("Val Keypoints Class Count: {}".format(val_dataset_keypoints.num_classes))
for i, info in enumerate(val_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=8.80s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
Train Keypoints Image Count: 64115
Train Keypoints Class Count: 2
  0. BG                                                
  1. person                                            
Val Keypoints Image Count: 2693
Val Keypoints Class Count: 2
  0. BG                                                
  1. person                                            


## Ceate Model

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
print("Loading weights from ", COCO_MODEL_PATH)
# model.keras_model.summary()

Tensor("input_image:0", shape=(None, 1024, 1024, 3), dtype=float32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[<tf.Tensor 'fpn_p2/BiasAdd:0' shape=(None, 256, 256, 256) dtype=float32>, <tf.Tensor 'fpn_p3/BiasAdd:0' shape=(None, 128, 128, 256) dtype=float32>, <tf.Tensor 'fpn_p4/BiasAdd:0' shape=(None, 64, 64, 256) dtype=float32>, <tf.Tensor 'fpn_p5/BiasAdd:0' shape=(None, 32, 32, 256) dtype=float32>, <tf.Tensor 'fpn_p6/MaxPool:0' shape=(None, 16, 16, 256) dtype=float32>]
rpn_class:  Tensor("rpn_class/concat:0", shape=(None, None, 2), dtype=float32)
rpn_bbox:  Tensor("rpn_bbox/concat:0", shape=(None, None, 4), dtype=float32)
x im fpn_classifier_graph:  (2, None, 7, 7, 256)
s im fpn_classifier_grapf:  (None, None, 8)
Loading weights from  C:\Users\LStue\Mask_RCNN_Humanpose\mask_rcnn_coco_humanpose.h5


In [ ]:
# Training - Stage 1
print("Train heads")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE,
            epochs=15,
            layers='heads')
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Training Resnet layer 4+")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=20,
            layers='4+')
# Training - Stage 3
# Finetune layers from ResNet stage 3 and up
print("Training Resnet layer 3+")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 100,
            epochs=100,
            layers='all')

Train heads

Starting at epoch 0. LR=0.002

Checkpoint Path: C:\Users\LStue\Mask_RCNN_Humanpose\mylogs\coco20201216T1535\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_keypoint_mask_conv1   (TimeDistributed)
mrcnn_keypoint_mask_bn1   (TimeDistributed)
mrcnn_keypoint_mask_conv2   (TimeDistributed)
mrcnn_keypoint_mask_bn2   (TimeDistributed)
mrcnn_keypoint_mask_conv3   (TimeDistributed)
mrcnn_keypoint_mask_bn3   (TimeDistributed)
mrcnn_keypoint_mask_conv4   (TimeDistributed)
mrcnn_keypoint_mask_bn4   (TimeDistributed)
mrcnn_keypoint_mask_conv5   (TimeDistributed)
mrcnn_keypoint_mask_bn5   (TimeD

D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15


D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\tensorflow_core\python\framew

   3/1000 [..............................] - ETA: 2:04:06 - loss: 7.3703 - rpn_class_loss: 0.0079 - rpn_bbox_loss: 0.1161 - mrcnn_class_loss: 0.5135 - mrcnn_bbox_loss: 0.5007 - keypoint_mrcnn_mask_loss: 4.9452 - mrcnn_mask_loss: 1.2869

D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


1000/1000 [==============================] - 1758s 2s/step - loss: 5.9345 - rpn_class_loss: 0.0126 - rpn_bbox_loss: 0.3083 - mrcnn_class_loss: 0.1165 - mrcnn_bbox_loss: 0.1684 - keypoint_mrcnn_mask_loss: 5.0786 - mrcnn_mask_loss: 0.2501 - val_loss: 5.5570 - val_rpn_class_loss: 0.0022 - val_rpn_bbox_loss: 0.3821 - val_mrcnn_class_loss: 0.0457 - val_mrcnn_bbox_loss: 0.1476 - val_keypoint_mrcnn_mask_loss: 4.7465 - val_mrcnn_mask_loss: 0.2328
Epoch 2/15
 138/1000 [===>..........................] - ETA: 24:03 - loss: 5.9033 - rpn_class_loss: 0.0121 - rpn_bbox_loss: 0.2990 - mrcnn_class_loss: 0.1194 - mrcnn_bbox_loss: 0.1657 - keypoint_mrcnn_mask_loss: 5.0618 - mrcnn_mask_loss: 0.2453